In [2]:
import requests 
from bs4 import BeautifulSoup
import csv

URL = "https://www.whatmobile.com.pk/"
r = requests.get(URL) 
soup = BeautifulSoup(r.content, 'html5lib') 

products = soup.findAll("li", attrs = {'class': 'product'})
mobiles = []

for product in products:
    mobile = {}
    mobile['Name'] = product.find('a',attrs = {'class': 'BiggerText'})['title']
    mobile['Price'] = product.a['title']
    mobile['Img'] = URL+product.img['src']
    mobiles.append(mobile)

print("Mobile Detials are Saved in csv file. Open file to view it.")
filename = 'mobiledetails.csv'
with open(filename, 'w') as f:
    w = csv.DictWriter(f,['Name','Price','Img'], dialect="excel")
    w.writeheader()
    w.writerows(mobiles)

Mobile Detials are Saved in csv file. Open file to view it.
